In [9]:
from random import randint
from math import gcd

def randuntil(lb, ub, until):
    while True:
        x = randint(lb, ub)
        if until(x): return x
        
def hexdump(xs):
    return ' '.join(f'{x:08X}' for x in xs)

def is_prime(n):
  if n == 2 or n == 3: return True
  if n < 2 or n%2 == 0: return False
  if n < 9: return True
  if n%3 == 0: return False
  r = int(n**0.5)
  # since all primes > 3 are of the form 6n ± 1
  # start with f=5 (which is prime)
  # and test f, f+2 for being prime
  # then loop by 6. 
  f = 5
  while f <= r:
    if n % f == 0: return False
    if n % (f+2) == 0: return False
    f += 6
  return True    

In [10]:
def keygen(p=None, q=None):
    if p is None: p = randuntil(2, 1e4-1, is_prime);                   print('p =', p)
    if q is None: q = randuntil(2, 1e4-1, is_prime);                   print('q =', q)
    assert p != q
    n = p*q;                                                          print('n =', n)
    ϕ = (p - 1)*(q - 1);                                              print('ϕ =', ϕ)
    e = randuntil(3, 2**32, lambda x: is_prime(x) and gcd(x, ϕ) == 1); print('e =', e)
    # przyspieszenie: rozszerzony alg. euklidesa
    d = pow(e, -1, ϕ);                                                print('d =', d)
    #d = randuntil(2, 2**32, lambda x: e*x % ϕ == 1);                  print('d =', d)
    return (e, n), (d, n)

In [11]:
def cipher(key, msg):
    p, n = key
    return [pow(x, p, n) for x in msg]

encrypt = lambda k, m: cipher(k, bytes(m, 'utf-8'))
decrypt = lambda k, m: bytes(cipher(k, m)).decode('utf-8')

In [12]:
pub, prv = keygen()

p = 6269
q = 1723
n = 10801487
ϕ = 10793496
e = 1619757017
d = 7499081


In [15]:
original = 'Tajny tekst do zaszyfrowania - oryginalna wersja 1'
assert len(bytes(original, 'utf-8')) == 50
print(original)

Tajny tekst do zaszyfrowania - oryginalna wersja 1


In [16]:
secret = encrypt(pub, original)
print('length =', len(secret), '\n'+'-'*11)
print(hexdump(secret))

length = 50 
-----------
00713F87 00493F99 00978C94 0033E3CF 001C06B9 0093F074 00469BB8 005B37BF 003871C0 0076A0A6 00469BB8 0093F074 00135B40 0010B12E 0093F074 00117A51 00493F99 0076A0A6 00117A51 001C06B9 0091398E 0085DFC6 0010B12E 000E31B4 00493F99 0033E3CF 0022508E 00493F99 0093F074 0080D49D 0093F074 0010B12E 0085DFC6 001C06B9 00581BF6 0022508E 0033E3CF 00493F99 00110FAB 0033E3CF 00493F99 0093F074 000E31B4 005B37BF 0085DFC6 0076A0A6 00978C94 00493F99 0093F074 0092594C


In [17]:
recevied = decrypt(prv, secret)
assert recevied == original
print(recevied)

Tajny tekst do zaszyfrowania - oryginalna wersja 1
